In [1]:
import urllib
import re
import time
import traceback
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
#from leek import get_consumer
#from py_log import get_logger
#from retrying import retry
import time
import os

In [2]:
import pickle

In [3]:
query_lst = [l.strip() for l in open('./questions.txt').readlines()]
len(query_lst)

query = query_lst[50]

#url_pattern = 'http://zhidao.baidu.com/search?lm=0&rn=10&pn=0&fr=search&ie=gbk&dyTabStr=MCw1LDQsMSw2LDMsNyw4LDIsOQ%3D%3D&word='
#url_pattern = 'https://zhidao.baidu.com/search?lm=0&rn=10&pn=0&fr=search&ie=gbk&word='
#url_pattern = 'https://zhidao.baidu.com/search?word='

#url = url_pattern + urllib.parse.quote(query.encode('gbk'))

#url = 'https://zhidao.baidu.com'

#headers = { 
#    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36' 
#}

#req = urllib.request.Request(url=url, headers=headers, method='GET')
#res = urllib.request.urlopen(req)

#res = requests.request('GET', url, headers=headers, timeout=10)

#res.status
#
#txt = res.read().decode('gbk')



query

'我使用了针灸/推拿/等方式治疗，可以报销么？'

In [4]:
def spider(query, pn, output_file):
    pn*=10
    keyword = quote(query.encode('gbk'))
    url = f'https://zhidao.baidu.com/search?lm=0&rn=10&pn={pn}&fr=search&ie=gbk&word={keyword}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'Cookie': 'BAIDUID=18581A93B7660CCE2EF45F5C9AF78D5D:FG=1; __yjs_duid=1_5a656be8e909a5ab1dc8e6c51a171ff51620382781035; BIDUPSID=18581A93B7660CCE2EF45F5C9AF78D5D; PSTM=1620390429; BDSFRCVID_BFESS=a1POJeC62mInKaverYnGo6QQ_RiR2Y7TH6ao_I6yuPDV6U_ZuVjhEG0PVf8g0KA-hILjogKK3gOTH4PF_2uxOjjg8UtVJeC6EG0Ptf8g0M5; H_BDCLCKID_SF_BFESS=JbPtoD_yfIvafJjYhKTMbJt3-xrHKI62aKDsQI3gBhcqEIL45fJa3-tpbq5xt-juJTQr2prSypT8fxbSj4Qo2x4EQUbyQT3Q5KoaafjSBp5nhMJ_3j7JDMP0qfAjhlcy523iob3vQpPMVhQ3DRoWXPIqbN7P-p5Z5mAqKl0MLPbtbb0xXj_0D6OLDGA8t6ks-DTbB4oHK--_qnTz-4L_5-_e-xQyetJyaR3EWhRvWJ5TMCoz-UoUjqD8K4JmKUuJMeJrBhRgQxKbShPC-tPaj5LFK4oehx7n365EBIO13l02Vbc9e-t2yU_VKHjut4RMW20e0h7mWIbmsxA45J7cM4IseboJLfT-0bc4KKJxbnLWeIJEjjCKj6OyjHLjJTPj-6TKWDP8Kb7VbPTuyUnkbfJBDl5QyMbltKTHKKQXBCb08hR2bPoAeM47yajK25o72TcdK43JbJ7is-8wy5rpQT8rbtFOK5OibCrWBPOTab3vOIOTXpO1jh8zBN5thURB2DkO-4bCWJ5TMl5jDh3Mb6ksD-Ftqj_OtJ-foDKbf-OKJt5kq4bohjPw5mc9BtQmJJu80JRgHRrqqRbT5xFM2RFBDlQNa-0fQg-q3R79fR6-SfoLbJba-JFT-n370x-jLnbOVn0MW-5DVp3gMtnJyUnybPnnBT3T3H8HL4nv2JcJbM5m3x6qLTKkQN3T-PKO5bRu_CF5JIPMbD0menJb5ICq5f7-etJXfKCDbp7F5l8-hR6VhxRH54LXWfTx255ILGONBx_-yC3xOKQphpnzjRF00-OptqQdyjTpBJnN3KJmSRL9bT3vLDuJytvB2-biWb7M2MbdJUJP_IoG2Mn8M4bb3qOpBtQmJeTxoUJ25DnJhhCGe4bK-TryDHtfJfK; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; H_PS_PSSID=34099_31254_33848_33773_33607; delPer=0; BAIDUID_BFESS=EC0B8CB2E5DB2C252FE4F1A4011CDAD8:FG=1; PSINO=2; BA_HECTOR=8ka520a1812g25a0u31gc1pgs0r; ZD_ENTRY=empty; Hm_lvt_6859ce5aaf00fb00387e6434e4fcc925=1621172136,1623254974; Hm_lpvt_6859ce5aaf00fb00387e6434e4fcc925=1623254974; shitong_key_id=2; ab_sr=1.0.1_OTBiZTEzY2RmMWM3MWM1MWNkMDE4MzIwNjllOTJkYmY2M2FiN2UyZGFmYzJjMzllY2FhY2MwMTBjZDI5NzAzZDAxYzVlMzQ2Y2I2YTQyMTkyMWIxNmRhOWUxMTg3ZWU5MGY4Zjc2MmYyOGU0YjIxZmM1ZWMwNzY0ZWYzYzk0MDc1YjFmMTI3ODUyZjg2NmYyNjQ2MDU3MjRjYjg1NjU2Mg==; shitong_data=7d3434b10335de90105aee880b6976138ba670eb5ab5dfec1ec7f6fdead89d6526d65dacb8a6ec84de87e8a3bf8d823c50a8175b16a3745ff826feb07592236f5b646786f633ff511100475de3f136c051560bb904a45f5b5fab2e3376f81eee20967d70e060f03de9793416fa71e0f37a7b315109d2dbfad098fad82b5313db; shitong_sign=86830a0e'
    }

    wb_data = requests.request("GET", url, headers=headers, timeout=10)
    wb_data.encoding = 'gbk'

    html = wb_data.text

    #new_html = html.encode('gbk')

    new_html = html.replace('<br />', '\n').replace('<p>', '\n').replace('</p>', '\n').replace('<br>', '\n')

    #save_path = f'{output_dir}/{query}_{pn}.html'
    save_path = output_file
    print(f'save {query} to {save_path}')
    with open(save_path, 'w') as f:
        ## TODO
        try:
            f.write(new_html)
        except:
            print(f'save {query} to {save_path} error!')
    soup = BeautifulSoup(new_html, 'html.parser')
    ret = []
    for li in soup.find_all('a', attrs={'class':'ti'}):
        ret.append(li.text)
    print(f'get {len(ret)}')
    return ret

def sleep(bound=3, mu=5, sigma=10):
    import time
    import random
    t = -1
    while t<bound:
        t = random.normalvariate(5, 10)
    time.sleep(t)

In [5]:
output_dir = './htmls/'

os.system(f'mkdir -p {output_dir}')

query_lst = sorted(query_lst)

In [6]:
results = pickle.load(open('./resutls.pkl', 'rb'))

In [7]:
len(results)

86

In [8]:
continue_n = 180

In [ ]:
for n, query in enumerate(query_lst):
    if n<continue_n:
        continue
    ret = {}
    for pn in [0,1,2,20,70]:
        output_file = f'{output_dir}/{n}_{pn}.html'
        a = spider(query, pn, output_file)
        ret[pn] = a
        sleep(1,1,10)
    results[query] = ret
    sleep(bound=10)

save 摩尔线程为什么能这么快就研制成功第一款芯片 to ./htmls//180_0.html
get 10
save 摩尔线程为什么能这么快就研制成功第一款芯片 to ./htmls//180_1.html
get 10
save 摩尔线程为什么能这么快就研制成功第一款芯片 to ./htmls//180_2.html
get 10
save 摩尔线程为什么能这么快就研制成功第一款芯片 to ./htmls//180_20.html
get 6
save 摩尔线程为什么能这么快就研制成功第一款芯片 to ./htmls//180_70.html
get 6
save 摩尔线程为开发者提供了哪些支持 to ./htmls//181_0.html
get 10
save 摩尔线程为开发者提供了哪些支持 to ./htmls//181_1.html
get 10
save 摩尔线程为开发者提供了哪些支持 to ./htmls//181_2.html
get 10
save 摩尔线程为开发者提供了哪些支持 to ./htmls//181_20.html
get 10
save 摩尔线程为开发者提供了哪些支持 to ./htmls//181_70.html
get 3
save 摩尔线程在哪些城市有分公司 to ./htmls//182_0.html
get 10
save 摩尔线程在哪些城市有分公司 to ./htmls//182_1.html
get 10
save 摩尔线程在哪些城市有分公司 to ./htmls//182_2.html
get 10
save 摩尔线程在哪些城市有分公司 to ./htmls//182_20.html
get 5
save 摩尔线程在哪些城市有分公司 to ./htmls//182_70.html
get 5
save 摩尔线程是什么时候成立的 to ./htmls//183_0.html
get 10
save 摩尔线程是什么时候成立的 to ./htmls//183_1.html
get 10
save 摩尔线程是什么时候成立的 to ./htmls//183_2.html
get 10
save 摩尔线程是什么时候成立的 to ./htmls//183_20.html
get 4
save 摩尔线程是什么时候成立的 to .

In [ ]:
for n, query in enumerate(query_lst):
    if n<103:
        continue
    if n>180:
        continue
        
    ret = {}
    for pn in [0,1,2,20,70]:
        output_file = f'{output_dir}/{n}_{pn}.html'
        a = spider(query, pn, output_file)
        ret[pn] = a
        sleep(1,1,10)
    results[query] = ret
    sleep(bound=10)

In [ ]:
continue_n = 0
for n, query in enumerate(query_lst):
    if n<continue_n:
        continue
    nquery = query.replace('摩尔线程', '')
    if nquery!=query:
        ret = {}
        for pn in [0,1,2,20,70]:
            output_file = f'{output_dir}/new_{n}_{pn}.html'
            a = spider(nquery, pn, output_file)
            ret[pn] = a
            sleep(1,1,10)
        results[query][nquery] = ret
        sleep(bound=10)